# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [35]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [36]:
sinasc_raw[sinasc_raw['munResNome'] == 'Município ignorado - RO']
ignorado = sinasc_raw.loc[1863]
sinasc_raw = sinasc_raw.drop(1863)

# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [37]:
idade_media_maes = sinasc_raw.groupby('munResNome')['IDADEMAE'].mean()
idade_media_pais = sinasc_raw.groupby('munResNome')['IDADEPAI'].mean()

#resultados
idade_media_maes.head()
idade_media_pais.head()

munResNome
Alta Floresta D'Oeste      29.548701
Alto Alegre dos Parecis    29.179104
Alto Paraíso               28.833333
Alvorada D'Oeste           30.757282
Ariquemes                  32.466667
Name: IDADEPAI, dtype: float64

### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [38]:
aniversario = '2019-01-10'
dados_aniversario = sinasc_raw[sinasc_raw['DTNASC'] == aniversario]

peso_medio_bebes = dados_aniversario.groupby(['SEXO', 'ESCMAE'])['PESO'].mean()
peso_medio_bebes

SEXO       ESCMAE         
Feminino   1 a 3 anos         3651.000000
           12 anos ou mais    3351.111111
           4 a 7 anos         3152.375000
           8 a 11 anos        3120.909091
Masculino  1 a 3 anos         3078.333333
           12 anos ou mais    3015.428571
           4 a 7 anos         3105.000000
           8 a 11 anos        3497.550000
Name: PESO, dtype: float64

### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [39]:
# nascimentos por município
nascimentos_por_municipio = sinasc_raw['munResNome'].value_counts()

# menos nascimentos
municipio_menos_nascimentos = nascimentos_por_municipio.idxmin()

# menos nascimentos filtrados
dados_municipio_menos = sinasc_raw[sinasc_raw['munResNome'] == municipio_menos_nascimentos]

# idade das mães nesse município
idade_maes_menos = dados_municipio_menos['IDADEMAE']
idade_media_maes_menos = idade_maes_menos.mean()
idade_maxima_maes_menos = idade_maes_menos.max()
idade_minima_maes_menos = idade_maes_menos.min()

# idade dos pais nesse município
idade_pais_menos = dados_municipio_menos['IDADEPAI']
idade_media_pais_menos = idade_pais_menos.mean()
idade_maxima_pais_menos = idade_pais_menos.max()
idade_minima_pais_menos = idade_pais_menos.min()

print(f"Município com menos nascimentos: {municipio_menos_nascimentos}")
print(f"Idade média das mães: {idade_media_maes_menos}, máxima: {idade_maxima_maes_menos}, mínima: {idade_minima_maes_menos}")
print(f"Idade média dos pais: {idade_media_pais_menos}, máxima: {idade_maxima_pais_menos}, mínima: {idade_minima_pais_menos}")

nascimentos_por_municipio.idxmin()

Município com menos nascimentos: Castanheiras
Idade média das mães: 27.28125, máxima: 39, mínima: 17
Idade média dos pais: 30.392857142857142, máxima: 43.0, mínima: 17.0


'Castanheiras'

### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [40]:
# filtrando os dados para o mês de março
dados_marco = sinasc_raw[sinasc_raw['DTNASC'].str.startswith('2019-03')]

# contando o número de nascimentos por município em março
nascimentos_marco_por_municipio = dados_marco['munResNome'].value_counts()

# município com mais nascimentos em março
municipio_mais_nascimentos_marco = nascimentos_marco_por_municipio.idxmax()

# filtrando os dados para o município com mais nascimentos em março
dados_municipio_mais_marco = dados_marco[dados_marco['munResNome'] == municipio_mais_nascimentos_marco]

# quantidade de filhos vivos nesse município
qtd_filhos_vivos_marco = dados_municipio_mais_marco['QTDFILVIVO']
qtd_media_filhos_vivos_marco = qtd_filhos_vivos_marco.mean()
qtd_maxima_filhos_vivos_marco = qtd_filhos_vivos_marco.max()
qtd_minima_filhos_vivos_marco = qtd_filhos_vivos_marco.min()

# idade dos pais nesse município
idade_pais_marco = dados_municipio_mais_marco['IDADEPAI']
idade_media_pais_marco = idade_pais_marco.mean()
idade_maxima_pais_marco = idade_pais_marco.max()
idade_minima_pais_marco = idade_pais_marco.min()

print(f"Município com mais nascimentos em março: {municipio_mais_nascimentos_marco}")
print(f"Idade média dos pais: {idade_media_pais_marco}, máxima: {idade_maxima_pais_marco}, mínima: {idade_minima_pais_marco}")

Município com mais nascimentos em março: Porto Velho
Quantidade média de filhos vivos: 1.2950558213716108, máxima: 9.0, mínima: 0.0
Idade média dos pais: 34.62903225806452, máxima: 62.0, mínima: 19.0


### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

Exemplo:
- Ah, descobri que a idade mediana das mulheres que deram a luz no ano de 2019 dos municipios x é maior que y.

* Percebi que as mães geralmente são mais novas que os pais, pelo menos nos municípios analisados. 
* Também, que na Capital o intervalo de idades dos pais é enorme (tem a partir de 19 anos, até 62).

In [41]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [42]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [43]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')